# Capstone Notebook 2: Further Analysis of NBA Shot Logs

# Introduction
- In this notebook we will look at some key offensive and defensive players
- We will look at the players with the highest close-range shooting percentages. We will limit the analysis to players who take at least as many shots as the league average
- We will look at players with the highest 3-point shooting accuracy. Again, with a minimum cut-off
- We will also look at the best mid-range shooters between 10 and 24 feet from the basket.
- We will look at the best defenders.  We will look at the percentage of shots missed while that defender is guarding.  With a minimum avg of 10 shots defended per game.
- This will further our previous analysis.  This will also give further evidence for the growing importance of the close-range shot and the 3-point shot in the NBA.
- We will also show the mid-range shot is not dead.  Just not the first line option it has been in the past

# 1.1 Overall View of NBA's Scorers And Defenders
- First view is total field goals made by each player. We will look closer at shot percentages close range and from the 3-point line.
- Second view is field goals made (at least 200) with the closest defender.  Usually a team’s best defenders guard the other team’s best shooters.  Consequently, the best defenders have a the most points scored against them.  We will analyze this in more detail by determining which defenders produce the highest percentage of missed shots when defending.
- Third view shows the overall point totals for the league’s top 12 scorers.  This view is important.  This view also includes points from free throws.  The rest of the data set only includes information on shots from the field.  It will be interesting to see how the best shooters differ from the best overall scorers.  Most of the leagues top scorers also shoot the most free-throws.  It should be similar but not exact.


In [1]:
%%HTML
<div class='tableauPlaceholder' id='viz1566933415918' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Sc&#47;ScorersandDefenders&#47;ScorersandDefenders&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='ScorersandDefenders&#47;ScorersandDefenders' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;Sc&#47;ScorersandDefenders&#47;ScorersandDefenders&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1566933415918');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.minWidth='920px';vizElement.style.maxWidth='1120px';vizElement.style.width='100%';vizElement.style.height='887px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# 1.2 Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import xticks
from scipy.stats import skew
import seaborn as sns

# 1.3 Read the NBA Shots Log Data

In [3]:
# Read The NBA Shots Log Data from csv file
data = pd.read_csv('shot_logs.csv')

In [4]:
# Visualize the head of the data frame
data.head(3)

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148


# 1.4 Top 10 Players by Overall Shooting Percentage

#### DETERMINE THE OVERALL SHOOTING PERCENTAGE OF ALL PLAYERS IN THE NBA

In [5]:
# Nba shot accuracy
made = data[data.SHOT_RESULT == 'made']['SHOT_RESULT'].count()
accuracy = round(made / len(data), 2)
print('The Overall Shooting Percentage For All NBA Players is {}%'.format(accuracy*100))

The Overall Shooting Percentage For All NBA Players is 45.0%


#### DETERMINE THE NUMBER OF SHOTS PER GAME FOR EACH NBA PLAYER

In [6]:
# Shots per Game
shot_taken = data.groupby(['player_name', 'GAME_ID'])['SHOT_RESULT'].count()
shot_taken = shot_taken.reset_index()
shot_taken = round(shot_taken.groupby('player_name')['SHOT_RESULT'].mean(),2)
shot_taken = shot_taken.reset_index()
shot_taken.columns = ['player_name', 'mean_shot']
shot_taken.head()

,player_name,mean_shot
0,aaron brooks,9.20
1,aaron gordon,3.85
2,al farouq aminu,4.78
3,al horford,12.54
4,al jefferson,16.00


In [7]:
print('The Average Number Of Shots Per Game In The NBA is {}'.format(round(shot_taken.mean_shot.mean(), 2)))

The Average Number Of Shots Per Game In The NBA is 8.73


#### WE WILL LIMIT OUR ANALYSIS TO PLAYERS WITH AT LEAST 9 SHOTS PER GAME

In [8]:
players_made = data[data.SHOT_RESULT == 'made'].groupby('player_name')['SHOT_RESULT'].count()
total_shots_player = data.groupby('player_name')['SHOT_RESULT'].count()
accuracy_players = round(players_made / total_shots_player,4) *100
accuracy_players = accuracy_players.reset_index()
accuracy_players.columns = ['player_name', 'accuracy']
accuracy_players['avg_shots_pg']  = shot_taken['mean_shot']
# Now filter for above 9 shots per game
accuracy_filtered_players = accuracy_players[accuracy_players.avg_shots_pg >= 9]
accuracy_filtered_players = accuracy_filtered_players.sort_values(['accuracy'], ascending=False)
accuracy_filtered_players.head()

,player_name,accuracy,avg_shots_pg
81,dwight howard,57.49,10.77
179,marcin gortat,54.56,9.34
71,derrick favors,54.51,12.09
19,anthony davis,54.34,17.16
3,al horford,54.13,12.54


#### HIGHEST SHOOTING PERCENTAGES OCCUR CLOSE TO THE BASKET.  ADD AVERAGE SHOT DISTANCE TO DATA. THIS WILL BE GOOD FOR COMPARISON TO OUR 3-POINT SHOOTING ANALYSIS

In [9]:
# Add avg shot distance, greater than 9 shots per game, keep top 10 players shooting percentage
mean_dist_shot = data.groupby('player_name')['SHOT_DIST'].mean()
mean_dist_shot = mean_dist_shot.reset_index()
mean_dist_shot.columns = ['player_name', 'mean_dist_shot']
accuracy_players['avg_shot_dist'] = mean_dist_shot['mean_dist_shot']
top_accuracy = accuracy_players[accuracy_players.avg_shots_pg >= 9]
top_accuracy = top_accuracy.sort_values(['accuracy'], ascending=False)
top_accuracy = top_accuracy[:10]
top_accuracy = top_accuracy.sort_values(['player_name'])
top_accuracy = top_accuracy.reset_index(drop=True)
top_accuracy

,player_name,accuracy,avg_shots_pg,avg_shot_dist
0,al horford,54.13,12.54,11.973566
1,andre drummond,50.83,11.08,4.400902
2,anthony davis,54.34,17.16,10.406540
3,carlos boozer,51.45,10.69,9.858387
4,derrick favors,54.51,12.09,7.729394
5,donatas motiejunas,50.87,9.76,9.441493
6,dwight howard,57.49,10.77,4.205689
7,marcin gortat,54.56,9.34,7.375614
8,nene hilario,52.40,9.04,9.740292
9,nikola vucevic,53.22,16.11,9.473503


## TOP 10 BEST SHOOTING PERCENTAGES (CLOSE RANGE)

In [10]:
%%HTML
<div class='tableauPlaceholder' id='viz1566935632399' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;CR&#47;CRShooters&#47;CRShooters&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='CRShooters&#47;CRShooters' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;CR&#47;CRShooters&#47;CRShooters&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1566935632399');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Conclusions:
- Not suprisingly all the players in the NBA with the highest shooting percentages are "Big Men". Players who play and take most of there shots close to the basket.  The Dunk is still the highest percentage shot in the game!

# 1.5 Top 15 Best Long Range 3-point Shooters


#### FILTER DATA FOR SHOTS LONGER THAN 24 FEET

In [11]:
# Filter for long range shots > than 24 feet
lr = data[data.SHOT_DIST >=24]

#### LONG RANGE SHOTS TAKEN PER GAME

In [12]:
# Get the avg number of LR 3's taken per player
lr_taken = lr.groupby(['player_name', 'GAME_ID'])['SHOT_RESULT'].count()
lr_taken = lr_taken.reset_index()
lr_taken = round(lr_taken.groupby('player_name')['SHOT_RESULT'].mean(),2)
lr_taken.head()

player_name
aaron brooks       3.07
aaron gordon       1.00
al farouq aminu    1.53
al horford         1.10
al jefferson       1.00
Name: SHOT_RESULT, dtype: float64

#### FIND LEAGUE AVG FOR LONG RANGE THREEs

In [13]:
# Find the League Average for Long range 3's
print("The League Avg For Long Range Three's is {}".format(lr_taken.mean()))

The League Avg For Long Range Three's is 2.2520717131474095


#### REFACTOR THE DATAFRAME AND FILTER FOR PLAYERS TAKING MORE THAN 3 LONG RANGE THREE's PER GAME

In [14]:
lr_taken = lr_taken.reset_index()
lr_taken.columns = ['player_name', 'avg_lr_shots']
lr_taken.head()

,player_name,avg_lr_shots
0,aaron brooks,3.07
1,aaron gordon,1.00
2,al farouq aminu,1.53
3,al horford,1.10
4,al jefferson,1.00


In [15]:
# Get the players shooting accuracy
players3_made = lr[lr.SHOT_RESULT == 'made'].groupby('player_name')['SHOT_RESULT'].count()
total_shots3_player = lr.groupby('player_name')['SHOT_RESULT'].count()
acc_players = round(players3_made / total_shots3_player,4) *100

In [16]:
# Refactor the dataframe
acc_players = acc_players.reset_index()
acc_players.columns = ['player_name', 'accuracy']
acc_players['avg_lr_shots_taken']  = lr_taken['avg_lr_shots']

In [17]:
# Filter the dataframe for more than 3 LR shots per game
best_accuracy3 = acc_players[acc_players.avg_lr_shots_taken > 3]
best_accuracy3 = best_accuracy3.sort_values(['accuracy'], ascending=False)

In [18]:
# List the top 15 Players
best_accuracy3 = best_accuracy3[:15]
best_accuracy3 = best_accuracy3.sort_values(['player_name'])
best_accuracy3 = best_accuracy3.reset_index(drop=True)
best_accuracy3

,player_name,accuracy,avg_lr_shots_taken
0,aaron brooks,40.70,3.07
1,brandon knight,41.43,4.40
2,channing frye,39.69,4.44
3,chris bosh,37.30,3.07
4,chris paul,39.34,3.46
5,danny green,39.34,3.64
6,james harden,38.04,5.88
7,joe johnson,37.87,3.13
8,klay thompson,42.86,5.16
9,kyle korver,46.38,3.57


## TOP 15 BEST LONG RANGE SHOOTERS

In [19]:
%%HTML
<div class='tableauPlaceholder' id='viz1566943320259' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;LR&#47;LRShooters&#47;LRShooters&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='LRShooters&#47;LRShooters' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;LR&#47;LRShooters&#47;LRShooters&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1566943320259');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Conclusions:
- Interesting to compare the top 12 overall scorers from the first visualization to the best short- and long-range scores.
- This is an important reason why I have included the number of shots attempted on the last two graphs.
- You often have specialists that shoot a fewer number of shots but do so at a high percentage.
- Then you have players like Steph Curry and Anthony Davis that are better all-around players. They attempt a larger number of shots, have a high percentage and have more free throw points.  Steph is at the top of the long-range shooters as well as overall scorers.  Similarly, Anthony Davis is among the leaders in short range shooting percentage and is also at the top of the overall scorers list.
- They both shoot a lot, make a lot, and get to the free throw line a lot.


# 1.6 Top 15 Best Defenders

#### FIND THE AVERAGE NUMBER OF SHOTS TAKEN AGAINST PLAYER PER GAME

In [20]:
# Average number of shots against defender per game
def_shots = data.groupby(['CLOSEST_DEFENDER', 'GAME_ID'])['SHOT_RESULT'].count()
def_shots = def_shots.reset_index()
def_shots = round(def_shots.groupby('CLOSEST_DEFENDER')['SHOT_RESULT'].mean(),2)
def_shots = def_shots.reset_index()
def_shots.columns = ['CLOSEST_DEFENDER', 'avg_shots_defended']
def_shots.head()

,CLOSEST_DEFENDER,avg_shots_defended
0,"Acy, Quincy",5.87
1,"Adams, Jordan",2.50
2,"Adams, Steven",9.68
3,"Adrien, Jeff",4.56
4,"Afflalo, Arron",7.90


#### BEST DEFENDERS DEFEND TOP SHOOTERS. FILTER BY MORE THAN 10 SHOTS PER GAME DEFENDED

In [21]:
# Define missed shot percentage against defender
defender_miss = data[data.SHOT_RESULT == 'missed'].groupby('CLOSEST_DEFENDER')['SHOT_RESULT'].count()
total_shots_defender = data.groupby('CLOSEST_DEFENDER')['SHOT_RESULT'].count()
accuracy_defenders = round(defender_miss / total_shots_defender,4) *100

In [22]:
# Combine missed shot percentage and average number of shots defended
accuracy_defenders = accuracy_defenders.reset_index()
accuracy_defenders.columns = ['CLOSEST_DEFENDER', 'Missed Shot %']
accuracy_defenders['avg_num_shots_defended']  = def_shots['avg_shots_defended']

In [23]:
# Filter for more than 10 shots defended
best_def = accuracy_defenders[accuracy_defenders.avg_num_shots_defended > 10]
best_def = best_def.sort_values(['Missed Shot %'], ascending=False)

In [24]:
# Display top 15 defenders
best_def = best_def[:15]
best_def = best_def.sort_values(['CLOSEST_DEFENDER'])
best_def

,CLOSEST_DEFENDER,Missed Shot %,avg_num_shots_defended
36,"Batum, Nicolas",58.73,10.02
49,"Bledsoe, Eric",56.92,10.66
90,"Collison, Darren",56.30,10.58
106,"Davis, Anthony",59.44,12.43
142,"Favors, Derrick",57.02,10.68
155,"Gasol, Marc",56.44,10.48
168,"Gortat, Marcin",56.10,11.28
172,"Green, Draymond",59.92,12.95
185,"Harden, James",56.51,10.29
205,"Holiday, Jrue",56.85,10.65


## TOP 15 BEST NBA DEFENDERS

In [25]:
%%HTML
<div class='tableauPlaceholder' id='viz1566950114707' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;To&#47;Top15Defenders&#47;Top15Defenders&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='Top15Defenders&#47;Top15Defenders' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;To&#47;Top15Defenders&#47;Top15Defenders&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1566950114707');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

## Conclusions:
- In the NBA the best defenders are usually tasked with guarding the other teams’ best players.  Players who take a lot of shots.  That is why I chose to include players who defend at least 10 shots per game.
- Not surprisingly the list includes many centers.  The “Big Men” who stay close to the basket and guard the rim.  
- What was surprising to me as Houston Rockets fan is James Harden, someone who gets a bad rap as a poor defensive player, is actually a well above average defensive player!


# 1.7 Top 15 Best Mid-Range Shooters
-They say the mid-range game is dead! I hypothesize Chris Paul and Dirk Nowtizski will be on the list

#### LOOK AT SHOTS BETWEEN 10 AND 24 FEET

In [26]:
mr = data[(data.SHOT_DIST >10) & (data.SHOT_DIST <25)]

#### MIDRANGE SHOTS TAKEN PER GAME

In [27]:
# Get the avg number mid-range shots taken per player
mr_taken = mr.groupby(['player_name', 'GAME_ID'])['SHOT_RESULT'].count()
mr_taken = mr_taken.reset_index()
mr_taken = round(mr_taken.groupby('player_name')['SHOT_RESULT'].mean(),2)
mr_taken.head()

player_name
aaron brooks       3.44
aaron gordon       1.72
al farouq aminu    2.48
al horford         6.45
al jefferson       5.98
Name: SHOT_RESULT, dtype: float64

#### LEAGUE AVERAGE FOR MID-RANGE SHOTS

In [28]:
# Find the League Average for Long range 3's
print("The League Avg for Mid-range Shots is {}".format(mr_taken.mean()))

The League Avg for Mid-range Shots is 4.363821428571429


#### CREATE DATA FRAME AND FILTER FOR MORE THAN 5 MID-RANGE SHOTS

In [29]:
mr_taken = mr_taken.reset_index()
mr_taken.columns = ['player_name', 'avg_mr_shots']
mr_taken.head()

,player_name,avg_mr_shots
0,aaron brooks,3.44
1,aaron gordon,1.72
2,al farouq aminu,2.48
3,al horford,6.45
4,al jefferson,5.98


In [30]:
# Get the players shooting accuracy
playersmr_made = mr[mr.SHOT_RESULT == 'made'].groupby('player_name')['SHOT_RESULT'].count()
total_shotsmr_player = mr.groupby('player_name')['SHOT_RESULT'].count()
acc_players_mr = round(playersmr_made / total_shotsmr_player,4) *100

In [31]:
# Combine player name, accuracy, and avg mr shots taken per game
acc_players_mr = acc_players_mr.reset_index()
acc_players_mr.columns = ['player_name', 'accuracy']
acc_players_mr['avg_mr_shots_taken']  = mr_taken['avg_mr_shots']

In [32]:
# Filter the dataframe for more than 5 MR shots per game
best_accuracy_mr = acc_players_mr[acc_players_mr.avg_mr_shots_taken > 5]
best_accuracy_mr = best_accuracy_mr.sort_values(['accuracy'], ascending=False)

In [33]:
# List the top 15 Players
best_accuracy_mr = best_accuracy_mr[:15]
best_accuracy_mr = best_accuracy_mr.sort_values(['player_name'])
best_accuracy_mr = best_accuracy_mr.reset_index(drop=True)
best_accuracy_mr

,player_name,accuracy,avg_mr_shots_taken
0,al horford,47.37,6.45
1,chris bosh,44.23,12.02
2,chris paul,47.84,10.08
3,courtney lee,46.40,6.20
4,darren collison,44.34,7.27
5,david west,44.24,7.33
6,dirk nowtizski,46.21,11.16
7,jason smith,43.87,5.53
8,jj redick,44.35,8.70
9,kyle korver,49.14,5.93


## NBA's TOP 15 MID-RANGE SHOOTERS (BETWEEN 10 AND 24 FT)

In [34]:
%%HTML
<div class='tableauPlaceholder' id='viz1566955265249' style='position: relative'><noscript><a href='#'><img alt=' ' src='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;MR&#47;MRShooters&#47;MRShooters&#47;1_rss.png' style='border: none' /></a></noscript><object class='tableauViz'  style='display:none;'><param name='host_url' value='https%3A%2F%2Fpublic.tableau.com%2F' /> <param name='embed_code_version' value='3' /> <param name='site_root' value='' /><param name='name' value='MRShooters&#47;MRShooters' /><param name='tabs' value='no' /><param name='toolbar' value='yes' /><param name='static_image' value='https:&#47;&#47;public.tableau.com&#47;static&#47;images&#47;MR&#47;MRShooters&#47;MRShooters&#47;1.png' /> <param name='animate_transition' value='yes' /><param name='display_static_image' value='yes' /><param name='display_spinner' value='yes' /><param name='display_overlay' value='yes' /><param name='display_count' value='yes' /><param name='filter' value='publish=yes' /></object></div>                <script type='text/javascript'>                    var divElement = document.getElementById('viz1566955265249');                    var vizElement = divElement.getElementsByTagName('object')[0];                    vizElement.style.width='100%';vizElement.style.height=(divElement.offsetWidth*0.75)+'px';                    var scriptElement = document.createElement('script');                    scriptElement.src = 'https://public.tableau.com/javascripts/api/viz_v1.js';                    vizElement.parentNode.insertBefore(scriptElement, vizElement);                </script>

# Conclusions
- In todays world, sport is guided by analytics, as much as any other business.
- Teams in the NBA, and all sports, employ large teams of people that analyze game data.
- These insights guide all aspects of the game.  From the offensive philosophy, to the players a team has on its roster.
- Team like the Houston Rockets have built their whole team philosophy around analytics.  Their offense is built around shooting either high percentage close range shots or the 3-pointer. The rockets attempt (excluding Chris Paul!) very few mid-range shots.  They view this shot as lower percentage with lower reward.  
- It was interesting to investigate the best close-range, mid-range, and long-range shooters.  It’s not surprising, that the people who showed up in more than one of the top categories, are some of the best players in the league.
- Because of this philosophy and approach the NBA is moving more towards teams with one or two great all around players surrounded by a supporting cast of role players and specialists. Players that shoot the three well or big men that play under the basket and take close range shots
